### Numba Vs Numpy

*Notes:*
- Cells need to be ran twice to avoid jit compilation overhead.

In [1]:
# Append rolling_ta to sys path.
import sys
import os

# Assuming rolling_ta is in the same level as tests/
module_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add the path to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from rolling_ta.env import NUMBA_DISK_CACHING
from rolling_ta.extras.numba import _empty, _shift, _prefix_sum, _mean
from rolling_ta.logging import logger
import numpy as np

from time import time

In [3]:
NUMBA_DISK_CACHING

False

In [4]:
sample_data = np.random.rand(1_000_000_000)
dummy_data = np.array([1,2,3], dtype=np.float64)

In [5]:
# Precompile
_empty(1_000_000, dtype=np.float64)
_empty(1_000_000, 1_000_000, dtype=np.float64)
_prefix_sum(dummy_data)
_shift(dummy_data)
_mean(dummy_data)

2.0

In [21]:
%%timeit
np.empty(1_000_000)

7.36 μs ± 223 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
%%timeit
_empty(1_000_000, dtype=np.float64)

48.3 μs ± 663 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [19]:
%%timeit
np.zeros(1_000_000, dtype=np.float64)

7.96 μs ± 255 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [20]:
%%timeit
_empty(1_000_000, 1_000_000)

1.55 ms ± 14.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%%timeit
sample_data.cumsum()

3.86 s ± 144 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
_prefix_sum(sample_data)

3.02 s ± 60.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
sample_data.mean()

1.33 s ± 38.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
_mean(sample_data)

292 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
np.roll(sample_data, -1)

1.65 s ± 56.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
_shift(sample_data)

1.65 s ± 42.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
